In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask import delayed, compute
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import lightgbm as lgb
import gc
from textblob import TextBlob, Word
import time
import pickle
from tqdm import tqdm,tqdm_pandas
path = "F:\\Pratik\\LearningZone\\Competitions\\Avito_Demand_Prediction_Challenge\\RawData"
os.chdir(path)

C:\Program Files\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
total_df = pd.read_csv('total_translated.csv', parse_dates=['activation_date'])

In [ ]:
# # Function to convert words from Russian to English:

# def translate(x):
#     try:
#         return TextBlob(x).translate(to="en")
#     except:
#         return x
    
# def conv_string(x):
#     try:
#         return translate(x).string
#     except:
#         return translate(x)
    
# @delayed
# def getting_title(total_df):
#     unique_reg_trans = []
#     unique_reg = list(pd.unique(total_df['title']))
#     for reg in unique_reg:
#         unique_reg_trans.append(conv_string(reg))

#     dict_title = dict(zip(unique_reg, unique_reg_trans)) # title Dictionary

#     total_df['en_title'] = total_df['title'].map(dict_title) # Mapping the titles in DF 
#     return total_df

# unique_reg_compute = getting_title(unique_reg)
# %time unique_reg_compute = unique_reg_compute.compute()

In [3]:
# Combining param1, param2 and param3 into 1 column:
total_df['combine_param1n2n3'] = total_df.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features
total_df.drop(["param_1","param_2","param_3"],axis=1,inplace=True)

In [8]:
# Extract features from text:
@delayed
def extract_textfeats(data):
    textfeats = ["description","combine_param1n2n3", "title"]
    for cols in textfeats:
        data[cols] = data[cols].astype(str) 
        data[cols] = data[cols].astype(str).fillna('nicapotato') # FILL NA
        data[cols] = data[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
        data[cols + '_num_chars'] = data[cols].apply(len) # Count number of Characters
        data[cols + '_num_words'] = data[cols].apply(lambda comment: len(comment.split())) # Count number of Words
        data[cols + '_num_unique_words'] = data[cols].apply(lambda comment: len(set(w for w in comment.split())))
        data[cols + '_words_vs_unique'] = data[cols+'_num_unique_words'] / data[cols+'_num_words'] * 100 # Count Unique Words
    return data

total_df = extract_textfeats(total_df)
%time total_df = total_df.compute()

In [25]:
# Scaling the price value:
total_df['price_new'] = np.log(total_df['price']+0.001)
total_df["price_new"].fillna(np.nanmean(total_df["price_new"].values), inplace=True)